### Imports

In [49]:
# import public packages
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys


from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram, hann, butter, filtfilt
from scipy import signal, interpolate
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif

### Add Directories / Load Files

In [25]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
ftg_path = find_folders.get_onedrive_path("FTG")
print(ftg_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\FTG_PROJECT


In [28]:
raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'raw_mats',
        "sub021",
        "sub-20210511PStn_ses-2022082612233578_run-BrainSense20220826125900.mat"
    ),
    info = None
)

sub = 'Sub021'

taskdict = {
    'RampUpThres': 1,
    'FastRamp': 2,
    '110Hz': 3,
    'Bilateral': 4,
    '145Hz': 5
}

Creating RawArray with float64 data, n_channels=6, n_times=230687
    Range : 0 ... 230686 =      0.000 ...   922.744 secs
Ready.


c:\Users\mathiopv\Anaconda3\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_14948\3061174646.py:1: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_14948\3061174646.py:1: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_14948\3061174646.py:1: RuntimeWarning: Cannot guess the correct type of channel LFP_S

#### Artefact Rejection & FIF Files

In [ ]:
#Interactive plot for artefact rejection
fig = raw.plot(n_channels = 2, highpass = 5, lowpass = 100, 
    filtorder = 5, duration = 20)
fig.fake_keypress('a')

In [ ]:
#First time making them
interactive_annot = raw.annotations
raw.annotations.save('Sub033_RampUpThres_artefactsAnnotations.csv', 
    overwrite = True)

In [37]:
#Importing them later
interactive_annot = pd.read_csv(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'clean_fifs',
        'sub021',
        'Sub021_RampUpThres_artefactsAnnotations.csv'
    )
)

#fix timing in imported annotations
reload(fix_annot_onsets)
new_onsets = fix_annot_onsets.fix_annot_onsets(interactive_annot)
print(new_onsets)

[125.9 148.9 177.2 204.5 301.7 347.  351.4 356.7 361.9 397.3 399.8 402.
 433.5 459.3 483.8 504.2 523.7 548.5 574.9 599.9 624.  654.2 656.7 679.3
 682.4 688.1 695.5 718.2 786.5 897. ]


In [47]:
my_annot = mne.Annotations(onset=new_onsets,  # in seconds
                           duration=interactive_annot.duration,  # in seconds, too
                           description=interactive_annot.description,
                           orig_time=raw.info['meas_date'])

reload(mat2fif)
%matplotlib qt
new_raw = mat2fif.mat2fif(raw,my_annot)

Setting 7271 of 230687 (3.15%) samples to NaN, retaining 223416 (96.85%) samples.
Creating RawArray with float64 data, n_channels=6, n_times=230687
    Range : 0 ... 230686 =      0.000 ...   922.744 secs
Ready.
Setting up band-pass filter from 5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 5.00, 100.00 Hz: -6.02, -6.02 dB



Channels marked as bad:
none


In [51]:
#Plot TF plot with artefact rejected data
reload(dat_preproc)
x = new_raw.get_data(reject_by_annotation = 'NaN',picks=[0,1])
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

NameError: name 'signal' is not defined